In [2]:
import re, json

# Define Design Space

In [3]:
def parse_verilog(verilog_file):
    with open(verilog_file, 'r') as f:
        content = f.read()
    
    module_name = re.search(r'module\s+(\w+)', content).group(1)
    parameters = re.findall(r'parameter\s+(\w+)\s*=\s*([\w\'"]+)', content)
    inputs = re.findall(r'input\s+(?:reg|wire)?\s*\[?([\w:]+)?\]?\s*(\w+)', content)
    outputs = re.findall(r'output\s+(?:reg|wire)?\s*\[?([\w:]+)?\]?\s*(\w+)', content)
    submodules = re.findall(r'(\w+)\s+#?\(?(?:\#?\([\w\s,=\']+\))?\s*(\w+)\s*\(([\w\s,.=]+)\);', content)
    
    module_info = {
        "module_name": module_name,
        "parameters": {param[0]: param[1] for param in parameters},
        "inputs": {inp[1]: inp[0] for inp in inputs},
        "outputs": {out[1]: out[0] for out in outputs},
        "submodules": []
    }

    for submodule in submodules:
        submodule_name, instance_name, connections = submodule
        connection_dict = {conn.split('(')[0].strip(): conn.split('(')[1].strip(' )') for conn in connections.split(',')}
        module_info["submodules"].append({
            "submodule_name": submodule_name,
            "instance_name": instance_name,
            "connections": connection_dict
        })
    
    return module_info

def generate_design_space(module_info):
    design_space = {
        "module": module_info["module_name"],
        "parameters": module_info["parameters"],
        "inputs": module_info["inputs"],
        "outputs": module_info["outputs"],
        "submodules": []
    }
    
    for submodule in module_info["submodules"]:
        design_space["submodules"].append({
            "submodule_name": submodule["submodule_name"],
            "instance_name": submodule["instance_name"],
            "connections": submodule["connections"],
            "customization_options": []  # Placeholder for customization options
        })
    
    return design_space

verilog_file = "alu.v"
alu_info = parse_verilog(verilog_file)
design_space = generate_design_space(alu_info)
print(json.dumps(design_space, indent=4))

{
    "module": "alu",
    "parameters": {
        "WIDTH": "32"
    },
    "inputs": {
        "H": "WIDT",
        "opcode": "3:0",
        "adder_sel": "1:0",
        "sub_sel": "1:0",
        "mul_sel": "1:0",
        "div_sel": "1:0",
        "shifter_sel": "1:0"
    },
    "outputs": {
        "H": "WIDT",
        "o": "zer"
    },
    "submodules": []
}


In [1]:
import itertools

# Define parameter values
subunits = ['adder', 'subtractor', 'multiplier', 'divider', 'shifter']
adder_types = ['ripple_carry', 'carry_lookahead']
bit_widths = [8, 16, 32]
pipeline_stages = [1, 2, 3]
clock_frequencies = [100e6, 200e6, 300e6]  # in Hz

# Generate all combinations
design_space = list(itertools.product(subunits, adder_types, bit_widths, pipeline_stages, clock_frequencies))

In [2]:
def encode_design_point(subunit, adder_type, bit_width, pipeline_stages, clock_frequency):
    return {
        'subunit': subunit,
        'adder_type': adder_type,
        'bit_width': bit_width,
        'pipeline_stages': pipeline_stages,
        'clock_frequency': clock_frequency
    }

# Encode all design points
encoded_design_space = [encode_design_point(*dp) for dp in design_space]

In [3]:
# Define a function to check constraints
def is_valid_design(design_point):
    # Example constraint: bit_width should be compatible with subunit type
    if design_point['subunit'] == 'adder' and design_point['adder_type'] == 'carry_lookahead' and design_point['bit_width'] > 16:
        return False
    return True

# Filter out invalid design points
valid_design_space = [dp for dp in encoded_design_space if is_valid_design(dp)]

# Explore Design Space

## Test Design

In [ ]:
import random

def simulate_design(design_point):
    # Code to run the testbench, collect timing, power, and performance data
    # Placeholder return value
    return random.random()

## Evaluate Design

In [ ]:
# Define evaluation function
def evaluate(individual):
    # Perform simulation and return performance metrics (e.g., latency, power)
    # Placeholder evaluation function
    performance_metric = simulate_design(individual)
    return performance_metric,

## DEAP

In [ ]:
from deap import base, creator, tools, algorithms

# Define fitness and individual
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", dict, fitness=creator.FitnessMin)

# Initialize toolbox
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, lambda: valid_design_space[random.randint(0, len(valid_design_space)-1)])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register genetic algorithm operations
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Run genetic algorithm
population = toolbox.population(n=100)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, stats=None, halloffame=None, verbose=True)

gen	nevals
0  	100   


C:\Users\ugheewala\Documents\sim\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\ugheewala\Documents\sim\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


TypeError: unhashable type: 'slice'

# Rank all Designs